In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import fastpathplanning as fpp

In [ ]:
L = np.array([
    [6.25,    4],
    [ 5.5,  2.5],
    [   1,    4],
    [ .25,  1.5],
    [2.25,  .75],
    [  .5, -.25],
    [   2,    0],
    [4.75, -.25],
    [   0,  5.2]
]) # lower bounds of the safe boxes
U = np.array([
    [7.25, 5.75],
    [ 7.5, 4.75],
    [   6,    5],
    [   7,    3],
    [   3,  2.5],
    [ 1.5,  4.5],
    [6.25,    1],
    [   6, 3.75],
    [   7,    6]
])# upper bounds of the safe boxes
p_init = np.array([1, .25]) # initial point
p_term = np.array([.5, 5.6]) # terminal point
T = 10 # traversal time
global_minimum = 0.5198965312777519
alpha = [0, 0, 1/global_minimum] # cost weights

In [ ]:
# segments in the mip parametrization
min_segments = 9
max_segments = 18
n_segments = max_segments - min_segments + 1
segments = np.arange(min_segments, max_segments + 1)

In [ ]:
# our times and cost
offline = 0.001501278604003906
polygonal = 0.0017126810527343748
smooth = 0.014897858378906226
ours = (offline + polygonal + smooth, 1.0001332093597541)
ours

In [ ]:
# mixed integer times and costs
times = []
costs = []
for N in segments:
    print(f'{N} segments:')
    path, cost, runtime = fpp.mixed_integer(L, U, alpha, p_init, p_term, T, N,
                                            solver='MOSEK', verbose=False)
    print(f'Solve time: {runtime}')
    print(f'Path cost: {cost}')

In [ ]:
mosek_times = [2.184000015258789, 6.04566502571106, 11.82934284210205, 12.819207191467285,
               15.553215026855469, 58.376124143600464, 123.59091114997864, 81.09816598892212,
               184.94813895225525, 404.21183705329895][:n_segments]
gurobi_times = [0.19363880157470703, 0.25190114974975586, 0.2583489418029785, 0.29549407958984375,
                0.37866711616516113, 0.5751729011535645, 0.6491379737854004, 0.7569220066070557,
                1.0049991607666016, 1.5620810985565186][:n_segments]
cplex_times = [0.12417006492614746, 0.1770470142364502, 0.19611287117004395, 0.27880215644836426,
               0.2930431365966797, 0.4250659942626953, 0.43905186653137207, 0.7057430744171143,
               0.7633371353149414, 0.8758621215820312][:n_segments]
costs = [1.2742344306549447, 1.2646842263720564, 1.1930159036048056, 1.0786720929559026,
         1.1753031445198519, 1.113692395115661, 1.0118039763104556, 1.0427954179079513,
         1.090919340997741, 1.0601955977954711][:n_segments]

In [ ]:
plt.rcParams.update({""
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": "Helvetica",
})

In [ ]:
plt.figure(figsize=(3.8, 3.5))

cplex_color = '#1f77b4'
gurobi_color = '#ff7f0e'
mosek_color = '#2ca02c'

plt.scatter(*ours, marker='*', color='yellow', edgecolor='k', s=150,
            label='FPP (ours)', zorder=5)
plt.plot(gurobi_times, costs, gurobi_color, marker='o',
         markeredgecolor='k', label='Gurobi')
plt.plot(cplex_times, costs, cplex_color, marker='o',
         markeredgecolor='k', label='CPLEX')
plt.plot(mosek_times, costs, mosek_color, marker='o',
         markeredgecolor='k', label='MOSEK')

props = dict(boxstyle='round', facecolor='white')
offset = 1.2
for i in range(len(cplex_times)):
    m = min([cplex_times[i], gurobi_times[i], mosek_times[i]])
    plt.plot([m, 2e3], [costs[i]] * 2, linestyle=':',
                 color='grey', zorder=-1)

plt.gca().set_xscale('log')
plt.grid()
plt.xlabel('Planning time (s)')
plt.ylabel('Path cost')
plt.gca().set_axisbelow(True)
plt.xlim([1e-2, 6e2])
ylims = [.98, 1.30]
plt.ylim(ylims)
# plt.legend(loc=2)
plt.gca().legend(loc='upper center', bbox_to_anchor=(0.48, 1.25), ncol=2)

ax2 = plt.gca().twinx()
ax2.set_ylabel('Subpaths $N$')
ax2.set_yticks(costs)
ax2.set_yticklabels([f'${i}$' for i in segments])
ax2.set_ylim(ylims)
plt.savefig('mip_comparison_revised.pdf', bbox_inches='tight')